In [1]:
!pip install wget
import os
import torch
import pandas as pd
import numpy as np
import torch
from torch import nn


ssh_config = """
Host github.com
  IdentityFile ~/.ssh/github.pem
  User davipeag
  StrictHostKeyChecking no
"""

if os.name == 'nt':
  base_path = ""
  REPO_DIR = "."
  STORE_DIR ="." 
  print("Windows")
else:
  print("Unix-like")
  REPO_DIR = "/tmp/HeartRateRegression"
  from google.colab import drive, auth
  drive.mount('/content/drive')
  GIT_PATH = "/content/drive/My\ Drive/deeplearning_project/github.pem"
  DATA_DIR = os.path.join(REPO_DIR, "repo")
  STORE_DIR ="/content/drive/My Drive/deeplearning_project/" 
  !mkdir ~/.ssh
  !cp -u {GIT_PATH} ~/.ssh/
  !chmod u=rw,g=,o= ~/.ssh/github.pem
  !echo "{ssh_config}" > ~/.ssh/config
  !chmod u=rw,g=,o= ~/.ssh/config
  ! (cd /tmp && git clone git@github.com:davipeag/HeartRateRegression.git)
  ! (cd {REPO_DIR} && git pull )
  import sys
  sys.path.append(REPO_DIR)

def git_pull():
  ! (cd {REPO_DIR} && git pull )

git_pull()

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=b40cdd03d53842631e3b020ec61c033f73cc841da1a8e8d2a04f5b1aff6be20a
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Unix-like
Mounted at /content/drive
Cloning into 'HeartRateRegression'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (265/265), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 1987 (delta 141), reused 156 (delta 57), pack-reused 1722
Receiving objects: 100% (1987/1987), 131.63 MiB | 28.65 MiB/s, done.
Resolving deltas: 100% (1286/1286), done.
Already up to date.
Already up to date.


In [2]:
import random
import torch
from torch import nn
import numpy as np
args = {
    'epoch_num': 250,     # Number of epochs.
    'lr': 1.0e-3,           # Learning rate.
    'weight_decay': 10e-4, # L2 penalty.
    'momentum': 0.9,      # Momentum.
    'num_workers': 0,     # Number of workers on data loader.
    'batch_size': 128,     # Mini-batch size. 128
    'batch_test': 248,     # size of test batch
    'window': 15,
    'initial_window':5,
    'clip_norm': 6.0,     # Upper limit on gradient L2 norm ###
}
if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

SEED = 1234
def reset_seeds():
  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

reset_seeds()

cuda


In [3]:
from data_utils import (Pamap2Handler, FormatPamap)

SUBJECTS = list(range(1,9))

handler = Pamap2Handler(DATA_DIR)
formatter = FormatPamap()
dfs_train = [formatter.transform(handler.get_protocol_subject(i)) for i in SUBJECTS]
[len(df)//200 for df in dfs_train]

download


[1882, 2235, 1264, 1647, 1873, 1809, 1567, 2040]

In [4]:
git_pull()

import importlib

import PPG
import preprocessing_utils
from PPG import FullTrainer
import RegressionHR

from RegressionHR import FullTrainer
from RegressionHR import PceLstmDefaults
from RegressionHR import PceLstmModel
from RegressionHR import TrainerJoint
from RegressionHR import  UtilitiesData
import Models
from  Models.BaseModels import ConstantHiddenSizeHalvingFullyConvolutionalEncoder1D
from Models import TimeSnippetModels

import PreprocessingHelpers

import Constants
from Constants import DatasetMapping


import Trainer
from  Trainer import BatchTrainers
from Trainer import BatchComputers
from Trainer import Interfaces

import CustomTrainers
from CustomTrainers import  DeepConvLstmFullTrainer
from CustomTrainers import SpecificTrainers, FullTrainers
importlib.reload(CustomTrainers)
importlib.reload(CustomTrainers.SpecificTrainers)
importlib.reload(CustomTrainers.FullTrainers)
import PreprocessingHelpers.TransformerGetters
                            
from PreprocessingHelpers import TransformerGetters

importlib.reload(Constants.DatasetMapping)

importlib.reload(PreprocessingHelpers)
importlib.reload(PreprocessingHelpers.TransformerGetters)
# importlib.reload(PPG.AttentionDefaults)
importlib.reload(PPG)
# importlib.reload(PPG.UtilitiesDataXY)
importlib.reload(PPG.Models)
importlib.reload(Models.BaseModels)
importlib.reload(Models.TimeSnippetModels)
# importlib.reload(PPG.NoHrPceLstmModel)
# importlib.reload(PPG.TrainerXY)
# importlib.reload(PPG.TrainerIS)
# importlib.reload(PPG.FullTrainer)
# importlib.reload(PceLstmDefaults)
# importlib.reload(preprocessing_utils)
importlib.reload(RegressionHR)
importlib.reload(RegressionHR.FullTrainer)
importlib.reload(RegressionHR.PceLstmDefaults)
importlib.reload(PPG.UtilitiesDataXY)
importlib.reload(preprocessing_utils)
importlib.reload(RegressionHR.TrainerJoint)
importlib.reload(RegressionHR.UtilitiesData)
importlib.reload(RegressionHR.PceLstmModel)
importlib.reload(preprocessing_utils)

importlib.reload(Trainer)
importlib.reload(Trainer.BatchTrainers)
importlib.reload(Trainer.BatchComputers)
importlib.reload(Trainer.ToolBox)
importlib.reload(Trainer.Interfaces )
# import imp
# for module in sys.modules.values():
#     importlib.reload(module)

Already up to date.


<module 'Trainer.Interfaces' from '/tmp/HeartRateRegression/Trainer/Interfaces.py'>

In [5]:
import matplotlib.pyplot as plt
import pickle

def compute_ensemble(results):
  
  ys = [v["predictions"][0].reshape(-1).numpy() for v in results]
  min_len_y = min([len(y) for y in ys])
  ys = [y[:min_len_y] for y in ys]
  for i in range(1, len(ys)-1):
    # assert np.all(ys[i] == ys[i-1])
    assert np.all(np.abs(ys[i] - ys[i-1])<1)
  ps = np.stack([v["predictions"][1].reshape(-1).numpy()[:min_len_y] for v in results])

  s = ps[0]
  for p in ps[1:]:
    s = s + p

  a = s/len(ps)
  y = ys[0]

  plt.figure()
  plt.plot(a)
  plt.plot(y)
  plt.show()

  return np.mean(np.abs(a - y)), np.mean(np.abs(ps - y))


def compute_ensemble_named(results, model_name="PceLstm"):
  
  results = [r[model_name] for r in results]

  ys = [v["labels"].reshape(-1) for v in results]
  min_len_y = min([len(y) for y in ys])
  ys = [y[:min_len_y] for y in ys]
  for i in range(1, len(ys)-1):
    # assert np.all(ys[i] == ys[i-1])
    assert np.all(np.abs(ys[i] - ys[i-1])<1)
  ps = np.stack([v["predictions"].reshape(-1)[:min_len_y] for v in results])

  s = ps[0]
  for p in ps[1:]:
    s = s + p

  a = s/len(ps)
  y = ys[0]

  plt.figure()
  plt.plot(a)
  plt.plot(y)
  plt.show()

  return np.mean(np.abs(a - y)), np.mean(np.abs(ps - y))

# compute_ensemble_named(aresults[0].values())

In [ ]:
fchoice = {
    "lr":0.001,
        "weight_decay": 0.0001,
        "batch_size": 16,
        "ts_sub": 0,
        "ts_per_window": 50,
        "ts_per_is": 12,
        "period_s": 3,
        }

discriminator_false_label = 0
from RegressionHR import FullTrainer2
import Trainer.BatchTrainers
from PPG import UtilitiesDataXY
from collections import defaultdict
import torch 
import CustomTrainers
from CustomTrainers import  DeepConvLstmFullTrainer

model_name = "DeepConvLSTM"
nepoch = 25
aresults = defaultdict(dict)
for val_sub in range(7):
  for ts_sub in range(8):
    fchoice['ts_sub'] = ts_sub
    full_trainer = DeepConvLstmFullTrainer(dfs_train, args["device"], nepoch, "pamap2",model_name)

    filename = f"deepconvlstm_{ts_sub}_{val_sub}_nepoch_{nepoch}.pkl"
    save_path = os.path.join(STORE_DIR, filename)
    try:
      with open(save_path , "rb") as f:
        out = pickle.load(f)
    except FileNotFoundError:
      out = full_trainer.train(**fchoice)
      with open(save_path, "wb") as f:
        pickle.dump(out, f)    
    print(out["args"], [out[k]['metric'] for k in (model_name,)])
    aresults[ts_sub][val_sub] = out
    print(f"{ts_sub}-TS:{compute_ensemble_named(list(aresults[ts_sub].values()), model_name)}")
  

best val epoch: 1
[1/100]: loss_train: [14.480724] loss_val [16.167648] loss_ts [14.961365]
best val epoch: 5
[5/100]: loss_train: [10.250943] loss_val [14.974679] loss_ts [18.616274]
best val epoch: 6
[6/100]: loss_train: [9.498102] loss_val [13.636551] loss_ts [20.192131]
best val epoch: 7
[7/100]: loss_train: [8.892305] loss_val [12.531017] loss_ts [15.060452]
best val epoch: 13
[13/100]: loss_train: [6.8147306] loss_val [11.594449] loss_ts [22.620295]
best val epoch: 14
[14/100]: loss_train: [7.4012203] loss_val [11.458854] loss_ts [16.820044]
best val epoch: 19
[19/100]: loss_train: [5.4211283] loss_val [10.653451] loss_ts [17.326351]
best val epoch: 20
[20/100]: loss_train: [5.850689] loss_val [10.537357] loss_ts [17.515783]
best val epoch: 33
[33/100]: loss_train: [5.6503677] loss_val [10.293064] loss_ts [20.20131]
best val epoch: 36
[36/100]: loss_train: [5.682698] loss_val [9.735205] loss_ts [17.702803]
